<a href="https://colab.research.google.com/github/myleskeller/ColabOffloading/blob/main/firstOrderMotionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Streamlined [Demo for paper "First Order Motion Model for Image Animation"](https://colab.research.google.com/github/AliaksandrSiarohin/first-order-model/blob/master/demo.ipynb)

In [ ]:
#@title Clone Repository / Import Python Requirements
import os.path
from os import path
# prevents recursively reloading repo if this cell is re-run
if(not(path.exists("/content/first-order-model"))):
  !git clone https://github.com/AliaksandrSiarohin/first-order-model
  %cd first-order-model

#@title Python Imports
import imageio
from skimage import img_as_ubyte
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

from demo import make_animation
from demo import load_checkpoints


def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

In [ ]:
#@title Load Models via Google Drive
#@markdown The **695 Mb** file `vox-cpk.pth.tar` contains the pre-trained models and is necessary for this to run. The simplest way (short of me pissing off GitHub by hosting it in a personal repo) is to add the [data folder associated with the original demo](https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing) (*same as link below*) to your Google Drive and link that to this runtime. This folder also contains example images and videos to experiment with. 

#@markdown * Add this folder to your Google Drive:
#@markdown https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing

#@markdown * Not working? Here's a mirror:
#@markdown https://drive.google.com/drive/folders/16inDpBRPT1UC0YMGMX3dKvRnOUsf5Dhn?usp=sharing

#@markdown 

# actual gogle drive stuff
from google.colab import drive
drive.mount('/content/drive')

#@markdown If the location of `vox-cpk.pth.tar` is not the default location, it can be changed below.
model_location = "/content/drive/MyDrive/first-order-motion-model/vox-cpk.pth.tar" #@param {type:"string"}

generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path=model_location)

# File Source
Choose one.

In [ ]:
#@title Direct Upload{ vertical-output: true }

#@markdown - In the **first** *Choose Files* prompt, select the desired `.jpg` or `.png` file to animate.
#@markdown - In the **second** *Choose Files* prompt, select the desired `.mp4` or `.mkv` file for animation modelling. 

from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  image_file_path = './' + fn
# input file path
print("Image file path: " + image_file_path)

uploaded = files.upload()
for fn in uploaded.keys():
  video_file_path = './' + fn
# input file path
print("Video file path: " + video_file_path)

In [ ]:
# @title Google Drive { vertical-output: true }

# actual gogle drive stuff
from google.colab import drive
drive.mount('/content/drive')

#@markdown - In the *Files* pane, find the desired `.jpg` or `.png` file to animate under the folder **drive**
#@markdown - click *Copy path* in the overflow menu of the file
#@markdown - paste the result in the `image_file_path` field below
# input image file path
image_file_path = '/content/drive/MyDrive/first-order-motion-model/cartoons-01.png' #@param {type: "string"}
print("Image file path: " + image_file_path)

#@markdown - In the *Files* pane, find the desired `.mp4` or `.mkv` file for animation modelling under the folder **drive**
#@markdown - click *Copy path* in the overflow menu of the file
#@markdown - paste the result in the `video_file_path` below
# input video file path
video_file_path = '/content/drive/MyDrive/first-order-motion-model/00.mp4' #@param {type: "string"}
print("Video file path: " + video_file_path)

# Render Animation

In [ ]:
#@title Configurable Parameters { form-width: "25%" }

keypoint_displacement = "absolute" #@param ["absolute", "relative"]
if keypoint_displacement == "absolute":
  keypoint_displacement = False
if keypoint_displacement == "relative":
  keypoint_displacement = True

adaptive_movement_scaling = True #@param {type:"boolean"}

source_image = imageio.imread(image_file_path)
driving_video = imageio.mimread(video_file_path, memtest=False)


#Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=keypoint_displacement, adapt_movement_scale=adaptive_movement_scaling)

HTML(display(source_image, driving_video, predictions).to_html5_video())